In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import col,to_date, year, month, unix_timestamp, lit, broadcast, concat, count, sum

In [ ]:
storage_account = "stworkshopbigdata"

In [ ]:
# Configure Storage Account connection with SAS Token (current Token valid until 22.06.2025)
#! This is Databricks config for running the jobs on one of the existing clusters
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account}.dfs.core.windows.net", "sp=racwdl&st=2023-06-21T16:43:59Z&se=2025-06-22T00:43:59Z&spr=https&sv=2022-11-02&sr=c&sig=0oCOUnwh%2B%2BoZzG39oOIVmNHm1HGvuwMz%2F8pyRSlVdOo%3D")

### 1.1 Read data for each year from Spark partitioned files

In [ ]:
spark_partitions_path = f"abfs://raw@{storage_account}.dfs.core.windows.net/spark-partitions-yellow"

Read data from Spark partitions for 2021 (30903722 rows) **28.62 seconds**

In [ ]:
spark.read.parquet(spark_partitions_path).filter("year == 2021").count() 

Read data from Spark partitions for 2020 (24648235 rows) **36.16 seconds**

In [ ]:
spark.read.parquet(spark_partitions_path).filter("year == 2020").count()

### 1.2 Read data for each year from manual prefix

In [ ]:
manual_partitions_path = f"abfs://raw@{storage_account}.dfs.core.windows.net/manual-prefix-yellow"

Read data from manual partitions for 2021 (30903722 rows) **55.90 seconds**

In [ ]:
spark.read.parquet(f"{manual_partitions_path}/2021/*").count()

Read data from manual partitions for 2021 (30903722 rows) **54.42 seconds**

In [ ]:
spark.read.parquet(f"{manual_partitions_path}/2020/*").count() 

### 2. Filter abfragen optimieren (optimaler filter pushdown)

##### *TASK* Get time series for a specific pickup location ID and time range: pickup from JFK Airport

JFK Airport location id = 132

No repartition, reading data from 1 month:
- from spark partitions: 1.23 min
- from manual partitions: 1.92 min

In [ ]:
# 1.23 minutes original partitions
spark.read.parquet(spark_partitions_path).select("pickup_date", "street_list", "PULocationID").filter((col("pickup_date").between("2021-01-01", "2021-01-02")) & (col("PULocationID") == 132)).count()

In [ ]:
# 1.91 minutes manual prefix
spark.read.parquet(f"{manual_partitions_path}/*/*").select("pickup_date", "street_list", "PULocationID").filter((col("pickup_date").between("2021-01-01", "2021-01-02")) & (col("PULocationID") == 132)).count()

In [ ]:
# 10.31 seconds
spark.read.parquet(f"{manual_partitions_path}/2021/04").filter((col("pickup_date").between("2021-04-01", "2021-05-01")) & (col("PULocationID") == 132)).count()

Repartition afrer reading

In [ ]:
partition_columns_1 = ["year", "pickup_date", "PULocationID"]

In [ ]:
df_repartitioned_1 = spark.read.parquet(f"{manual_partitions_path}/2021/04").repartition(*partition_columns_1)

In [ ]:
#50.61 seconds w/o previous cache, 33.87 seconds cached
df_repartitioned_1.filter((col("pickup_date").between("2021-04-01", "2021-04-30")) & (col("PULocationID") == 132)).count()

In [ ]:
partition_columns_2 = ["pickup_date", "PULocationID"]

In [ ]:

df_repartitioned_2 = spark.read.parquet(f"{manual_partitions_path}/2021/04").repartition(*partition_columns_2)

In [ ]:
# 1.39 minutes w/o previous cache 56.14 seconds cached
df_repartitioned_2.filter((col("pickup_date").between("2021-04-01", "2021-04-30")) & (col("PULocationID") == 132)).count()

In [ ]:
partition_columns_3 = ["PULocationID", "pickup_date"]

In [ ]:
df_repartitioned_3 = spark.read.parquet(f"{manual_partitions_path}/2021/04").repartition(*partition_columns_3)

In [ ]:
# 1.19 minutes w/o previous cache 1.22 cached
df_repartitioned_3.filter((col("pickup_date").between("2021-04-01", "2021-04-30")) & (col("PULocationID") == 132)).count()

In [ ]:
partition_columns_4 = ["PULocationID"]

In [ ]:
df_repartitioned_4 = spark.read.parquet(f"{manual_partitions_path}/2021/04").repartition(*partition_columns_4)

In [ ]:
# 1.31 minutes w/o previous cache 8.43 seconds cached
df_repartitioned_4.filter((col("pickup_date").between("2021-04-01", "2021-04-30")) & (col("PULocationID") == 132)).count()

#### Salting

In [ ]:
salted_df = spark.read.parquet(f"{manual_partitions_path}/2021/04").withColumn("partition_key", concat(col("pickup_date"), col("PULocationID")))

In [ ]:
salted_df_repartitioned = salted_df.repartition("partition_key")

In [ ]:
# 8.61 seconds
salted_df_repartitioned.filter((col("pickup_date").between("2021-04-01", "2021-04-30")) & (col("PULocationID") == 132)).count()

### 3. Join abfragen optimieren (optimale join keys evt. Broadcast join)

In [ ]:
df_zones = spark.read.format("csv") \
                .option("inferSchema", "true") \
                .option("header", "true") \
                .option("sep", ",") \
                .load(f"abfs://raw@{storage_account}.dfs.core.windows.net/taxi_zone_lookup.csv")

In [ ]:
df = spark.read.parquet(spark_partitions_path)

In [ ]:
# join without repartitions, regular join: 1.46 minutes
df_w_zones = df.join(df_zones, df["PULocationID"] == df_zones["LocationID"], "left")
df_w_zones.count()

In [ ]:
# join without repartitions, broadcast join: 1.47 minutes
df_w_zones_broadcast = df.join(broadcast(df_zones), df["PULocationID"] == df_zones["LocationID"], "left")
df_w_zones_broadcast.count()

In [ ]:
df_repartitioned = df.repartition("PULocationID")

In [ ]:
# Taxi data repartitioned by pickup ID: 1.77 minutes
df_w_zones = df_repartitioned.join(df_zones, df_repartitioned["PULocationID"] == df_zones["LocationID"], "left")
df_w_zones.count()

In [ ]:
df_zones_repartitioned = df_zones.repartition("LocationID")
df_zones_repartitioned.cache().count()

In [ ]:
# both data frames repartitioned by loc ID: 1.76 minutes
df_repartitioned.join(df_zones_repartitioned, df_repartitioned["PULocationID"] == df_zones_repartitioned["LocationID"], "left").count()

#### 3.1 Broadcast join on a small payment type table

In [ ]:
payment_type_data =  [
    (1, "Credit card"),
    (2, "Cash"),
    (3, "No charge"),
    (4, "Dispute"),
    (5, "Unknown"),
    (6, "Voided trip")
]

payment_type_schema = ["code","payment_type"]
df_payment_types = spark.createDataFrame(data=payment_type_data, schema = payment_type_schema)

In [ ]:
df_payment_types.display()

In [ ]:
df = spark.read.parquet(spark_partitions_path)

In [ ]:
df_w_payment = df.join(df_payment_types, df["payment_type"] == df_payment_types["code"], "left")

In [ ]:
# Regular join: 1.49 minutes
df_w_payment.count()

In [ ]:
df_broadcast = df.join(broadcast(df_payment_types), df["payment_type"] == df_payment_types["code"], "left")

In [ ]:
# Broadcast join: 1.49 minutes
df_broadcast.count()

In [ ]:
#1.66 minutes
df.repartition("payment_type").join(broadcast(df_payment_types), df["payment_type"] == df_payment_types["code"], "left").count()

### 4. GroupBy Aggregationen optimieren

##### 4.1. Total trips by Pickup Location in 2021

In [ ]:
df_2021 = spark.read.parquet(spark_partitions_path).filter("year == 2021")

In [ ]:
partitioned_df = df_2021.repartition("PULocationID")

In [ ]:
partitioned_df.cache()

In [ ]:
total_trips = partitioned_df.groupBy("PULocationID").agg(count("*").alias("total_trips"))

In [ ]:
df_2021.groupBy("PULocationID").agg(count("*").alias("total_trips")).limit(20).display()

In [ ]:
total_trips.limit(20).display() #2.69 min

For writing the data frame we can also use bucketing to optymize the write
https://luminousmen.com/post/the-5-minute-guide-to-using-bucketing-in-pyspark